# Caso de Negocio -- Scholastic Travel Company

La agencia de viajes Scholastic Travel Company desea crear una estrategia de retención de clientes. Para esto, es necesario que el equipo de analítica cree un modelo de predicción para determinar qué clientes de los que viajaron el año anterior (2011) van a reservar nuevamente el año siguiente (2012). 

En este Notebook se trabajarán los siguientes temas: 

- Separación de Datos en Train/Val/Test
- Análisis de Correlaciones
- Ajuste del mejor SVM 
- Métricas de Evaluación


<table class="tfo-notebook-buttons" align="center">

  <td>
    <a target="_blank" href="https://colab.research.google.com/github/juancop/metodos_analitica_2/blob/main/02_SVM/02_caso_de_negocio/02_caso_de_negocio.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" /></a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/juancop/metodos_analitica_2/blob/dev/01_optimization/02_SVM/02_caso_de_negocio/02_caso_de_negocio.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /></a>
  </td>
  
</table>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC

A continuación se carga la base de datos de STC, que cuenta con 2389 registros de clientes que viajaron en 2011 y tiene una variable que determina si el cliente viajó nuevamente en 2012. 

In [2]:
df = pd.read_csv('STC_datos_grupos.csv', sep = ';')

In [3]:
df.shape

(2389, 56)

In [4]:
df.head()

,ID,Program.Code,From.Grade,To.Grade,Group.State,Is.Non.Annual.,Days,Travel.Type,Departure.Date,Return.Date,...,GroupGradeTypeLow,GroupGradeTypeHigh,GroupGradeType,MajorProgramCode,SingleGradeTripFlag,FPP.to.School.enrollment,FPP.to.PAX,Num.of.Non_FPP.PAX,SchoolSizeIndicator,Retained.in.2012.
0,1,HS,4.0,4.0,CA,0,1,A,40557,40557,...,K,Elementary,K->Elementary,H,1,0.06364617,0.936508,4,L,1
1,2,HC,8.0,8.0,AZ,0,7,A,40557,40564,...,Middle,Middle,Middle->Middle,H,1,0.025882353,0.880000,3,L,1
2,3,HD,8.0,8.0,FL,0,3,A,40558,40560,...,Middle,Middle,Middle->Middle,H,1,0.02513089,0.888889,3,L,1
3,4,HN,9.0,12.0,VA,1,3,B,40558,40560,...,Undefined,Undefined,Undefined->Undefined,H,0,NaN,1.000000,0,NaN,0
4,5,HD,6.0,8.0,FL,0,6,T,40559,40564,...,Middle,Middle,Middle->Middle,H,0,0.1125,0.910112,8,M-L,0


Se cuenta con 54 variables independientes y una variable explicativa, `Retained.in.2012.`. 

In [5]:
df = df.drop(columns = ['FirstMeeting', 'LastMeeting', 'Initial.System.Date', 'Early.RPL', 'Latest.RPL', 'Departure.Date', 'Return.Date', 'Deposit.Date', 'ID'])

In [6]:
df.columns

Index(['Program.Code', 'From.Grade', 'To.Grade', 'Group.State',
       'Is.Non.Annual.', 'Days', 'Travel.Type', 'Special.Pay', 'Tuition',
       'FRP.Active', 'FRP.Cancelled', 'FRP.Take.up.percent.', 'Cancelled.Pax',
       'Total.Discount.Pax', 'Poverty.Code', 'Region', 'CRM.Segment',
       'School.Type', 'Parent.Meeting.Flag', 'MDR.Low.Grade', 'MDR.High.Grade',
       'Total.School.Enrollment', 'Income.Level', 'EZ.Pay.Take.Up.Rate',
       'School.Sponsor', 'SPR.Product.Type', 'SPR.New.Existing', 'FPP',
       'Total.Pax', 'SPR.Group.Revenue', 'NumberOfMeetingswithParents',
       'DifferenceTraveltoFirstMeeting', 'DifferenceTraveltoLastMeeting',
       'SchoolGradeTypeLow', 'SchoolGradeTypeHigh', 'SchoolGradeType',
       'DepartureMonth', 'GroupGradeTypeLow', 'GroupGradeTypeHigh',
       'GroupGradeType', 'MajorProgramCode', 'SingleGradeTripFlag',
       'FPP.to.School.enrollment', 'FPP.to.PAX', 'Num.of.Non_FPP.PAX',
       'SchoolSizeIndicator', 'Retained.in.2012.'],
      dtype=

# Limpieza de Datos

Nos entregan una base de datos que tiene demasiadas variables categóricas que pueden o no aportar información para el modelo. A continuación se limpiarán las categorías y se hará imputación de datos en caso de que haya variables faltantes. 

In [7]:
## Asignación de tipos categóricos

categorical_features = ['From.Grade', 'To.Grade', 'Group.State', 'Travel.Type', 'Poverty.Code', 'CRM.Segment', 'School.Type', 'Parent.Meeting.Flag', 
                        'MDR.High.Grade', 'School.Sponsor', 'SchoolGradeTypeLow', 'SchoolGradeTypeHigh', 'GroupGradeTypeLow', 'GroupGradeTypeHigh',
                        'MajorProgramCode', 'SingleGradeTripFlag', 'SchoolSizeIndicator', 'Region', 'Special.Pay',    
                        'Income.Level', 'SPR.Product.Type', 'SPR.New.Existing', 'DepartureMonth', 'MDR.Low.Grade', 
                        'FPP.to.School.enrollment', 'GroupGradeType', 'Program.Code', 'SchoolGradeType']

df[categorical_features] = df[categorical_features].apply(lambda x: x.astype('category'))



## División en Train y Test

Antes de comenzar con la limpieza de datos (que es inputación de missing values, estandarizaciones, etc...) es importante realizar la división en train-test. No queremos codificar en los datos de entrenamiento información de los datos de prueba... 

Para esto, utilice la función de [`sklearn.model_selection.train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

In [8]:
from sklearn.model_selection import train_test_split

y = df['Retained.in.2012.']
X = df.drop(columns = 'Retained.in.2012.')

### Utilice train_test_split para dividir los datos en entrenamiento y test. 

# Deje que el tamaño de test sea de 30%
# Random Sate: 42 para fines comparativos

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
                                                    
###

Verifique que la concentración de la variable dependiente sea similar en la base de entrenamiento y en la de validación.

In [9]:
y_test.value_counts()/len(y_test)

1    0.589958
0    0.410042
Name: Retained.in.2012., dtype: float64

In [10]:
y_train.value_counts()/len(y_train)

1    0.614833
0    0.385167
Name: Retained.in.2012., dtype: float64

## Missing Values

A continuación se realizará la inputación de valores faltantes. Utilizando el método `X_train.isna().sum()`

In [11]:
pd.DataFrame(X_train.isna().sum()/len(X_train))

,0
Program.Code,0.000000
From.Grade,0.054426
To.Grade,0.064593
Group.State,0.000000
Is.Non.Annual.,0.000000
Days,0.000000
Travel.Type,0.000000
Special.Pay,0.805622
Tuition,0.000000
FRP.Active,0.000000


Se observa que hay pocas columnas con valores faltantes. El 14% de los datos de entrenamiento no tuvo reuniones de padres, y el 25% no tiene asignado un código de pobreza. Es importante inputar los valores faltantes para poder realizar el entrenamiento. 

Utilice el método de [`sklearn.impute.SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) para entrenar un imputador para variables categóricas y variables numéricas. 

In [12]:
from sklearn.impute import SimpleImputer

categorical_columns = X_train.select_dtypes(exclude = [int, float]).columns
numerical_columns = X_train.select_dtypes(include = [int, float]).columns

### Imputador de Variables Categóricas -- strategy -> constant

categorical_imputer =  SimpleImputer(missing_values=np.nan, strategy='most_frequent')
###

In [13]:
### Imputador de Variables Numéricas -- strategy -> Median
numerical_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
###

In [14]:
categorical_imputer.fit(X_train[categorical_columns])

SimpleImputer(strategy='most_frequent')

In [15]:
numerical_imputer.fit(X_train[numerical_columns])

SimpleImputer()

In [16]:
# Utilice los imputers para inputar valores faltantes tanto en train como en test

###
X_train[categorical_columns] = categorical_imputer.transform(X_train[categorical_columns])
X_test[categorical_columns] = categorical_imputer.transform(X_test[categorical_columns])

X_train[numerical_columns] = numerical_imputer.transform(X_train[numerical_columns])
X_test[numerical_columns] = numerical_imputer.transform(X_test[numerical_columns])
###

In [17]:
# Verifique que efectivamente no tenga valores faltantes

###
pd.DataFrame(X_train.isna().sum()/len(X_train))

###

,0
Program.Code,0.0
From.Grade,0.0
To.Grade,0.0
Group.State,0.0
Is.Non.Annual.,0.0
Days,0.0
Travel.Type,0.0
Special.Pay,0.0
Tuition,0.0
FRP.Active,0.0


## Revisión Categorías

En la base de datos existe un gran número de variables categóricas. Se debe decidir de qué manera codificar la información (si requiere) para que el modelo pueda utilizarla. 

En principio, note que hay variables en las que hay categorías con pocas observaciones. Por ejemplo, la variable `From.Grade` tiene los siguientes conteos:


|               |   From.Grade |
|:--------------|-------------:|
| 8.0           |          786 |
| 7.0           |          365 |
| 6.0           |          159 |
| 4.0           |          109 |
| missing_value |           91 |
| 5.0           |           67 |
| 9.0           |           47 |
| 11.0          |           19 |
| 10.0          |           19 |
| 12.0          |            5 |
| 3.0           |            5 |

Se puede apreciar que la categoría 3 y 12 tienen 5 observaciones. Lo ideal es agrupar las categorías que tienen menos observaciones en una nueva categoría como "Otra".

In [18]:
for column in categorical_columns:
  categorical_counts = pd.value_counts(X_train[column])
  grouper = (categorical_counts/categorical_counts.sum() * 100).lt(2) # Selecciona aquellas con < 2% de observaciones
  X_train[column] = np.where(X_train[column].isin(categorical_counts[grouper].index),'Otra',X_train[column]) # Reemplaza en columna original
  X_test[column] = np.where(X_test[column].isin(categorical_counts[grouper].index),'Otra',X_test[column])

In [19]:
X_train['From.Grade'].value_counts()

8.0     877
7.0     365
6.0     159
4.0     109
5.0      67
Otra     48
9.0      47
Name: From.Grade, dtype: int64

## Variables Dummies

Algunas de las variables categóricas necesitan ser codificadas para poder entrar al modelo. Particularmente aquellas categorías que no son ordinales, como los estados donde se ubican los colegios. Por esta razón, utilizaremos One-Hot Encoding, para volver una variable categórica en un conjunto de variables binarias.

Utilice el método [`pd.get_dummies`](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) para extraer las dummies de variables categóricas.


<div class="alert alert-block alert-danger">
<b>Cuidado:</b> Utilizar un número excesivo de variables dummies puede afectar negativamente el desempeño del modelo.
</div>

In [20]:
X_train.head()

,Program.Code,From.Grade,To.Grade,Group.State,Is.Non.Annual.,Days,Travel.Type,Special.Pay,Tuition,FRP.Active,...,DepartureMonth,GroupGradeTypeLow,GroupGradeTypeHigh,GroupGradeType,MajorProgramCode,SingleGradeTripFlag,FPP.to.School.enrollment,FPP.to.PAX,Num.of.Non_FPP.PAX,SchoolSizeIndicator
380,HD,7.0,8.0,WA,0.0,4.0,A,FR,1912.0,4.0,...,March,Middle,Middle,Middle->Middle,H,0,Otra,0.888889,1.0,M-L
226,HD,8.0,8.0,CO,0.0,4.0,A,FR,1539.0,7.0,...,March,Middle,Middle,Middle->Middle,H,1,Otra,0.875000,2.0,S
1073,HD,8.0,8.0,CA,0.0,7.0,A,FR,2754.0,19.0,...,May,PK,Middle,PK->Middle,H,1,Otra,0.945946,2.0,S
715,HC,8.0,8.0,CA,0.0,6.0,A,FR,2549.0,30.0,...,April,Middle,Middle,Middle->Middle,H,1,Otra,0.882353,6.0,L
1195,HD,8.0,8.0,OH,0.0,6.0,B,SA,869.0,0.0,...,May,Elementary,Middle,Elementary->Middle,H,1,Otra,0.912409,12.0,M-L


In [28]:
pd.get_dummies(X_train[categorical_columns])

c:\Users\leonardo.garcia\Anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


,Program.Code_CD,Program.Code_HC,Program.Code_HD,Program.Code_HN,Program.Code_HS,Program.Code_HVP,Program.Code_Otra,Program.Code_SC,Program.Code_SK,From.Grade_4.0,...,MajorProgramCode_Otra,MajorProgramCode_S,SingleGradeTripFlag_0,SingleGradeTripFlag_1,FPP.to.School.enrollment_0.1,FPP.to.School.enrollment_Otra,SchoolSizeIndicator_L,SchoolSizeIndicator_M-L,SchoolSizeIndicator_S,SchoolSizeIndicator_S-M
380,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,1,0,0
226,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,1,0
1073,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,1,0
715,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0
1195,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1
1095,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0
1130,0,0,0,0,1,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,1
1294,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,1,0,0


In [21]:
###
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test) # Pero igual debemos asegurarnos que no haya "dummies de más"
###

c:\Users\leonardo.garcia\Anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
c:\Users\leonardo.garcia\Anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [22]:
X_train, X_test = X_train.align(X_test, fill_value = 0, axis = 1, join = 'left') # Alineamos datos para mantener mismas columnas

In [23]:
X_train.head()

,Is.Non.Annual.,Days,Tuition,FRP.Active,FRP.Cancelled,FRP.Take.up.percent.,Cancelled.Pax,Total.Discount.Pax,Total.School.Enrollment,EZ.Pay.Take.Up.Rate,...,MajorProgramCode_Otra,MajorProgramCode_S,SingleGradeTripFlag_0,SingleGradeTripFlag_1,FPP.to.School.enrollment_0.1,FPP.to.School.enrollment_Otra,SchoolSizeIndicator_L,SchoolSizeIndicator_M-L,SchoolSizeIndicator_S,SchoolSizeIndicator_S-M
380,0.0,4.0,1912.0,4.0,1.0,0.500,1.0,1.0,772.0,0.750,...,0,0,1,0,0,1,0,1,0,0
226,0.0,4.0,1539.0,7.0,2.0,0.500,3.0,2.0,230.0,0.286,...,0,0,0,1,0,1,0,0,1,0
1073,0.0,7.0,2754.0,19.0,1.0,0.543,1.0,2.0,250.0,0.114,...,0,0,0,1,0,1,0,0,1,0
715,0.0,6.0,2549.0,30.0,5.0,0.667,3.0,6.0,1027.0,0.289,...,0,0,0,1,0,1,1,0,0,0
1195,0.0,6.0,869.0,0.0,0.0,0.000,9.0,12.0,790.0,0.000,...,0,0,0,1,0,1,0,1,0,0


In [24]:
X_test.head()

,Is.Non.Annual.,Days,Tuition,FRP.Active,FRP.Cancelled,FRP.Take.up.percent.,Cancelled.Pax,Total.Discount.Pax,Total.School.Enrollment,EZ.Pay.Take.Up.Rate,...,MajorProgramCode_Otra,MajorProgramCode_S,SingleGradeTripFlag_0,SingleGradeTripFlag_1,FPP.to.School.enrollment_0.1,FPP.to.School.enrollment_Otra,SchoolSizeIndicator_L,SchoolSizeIndicator_M-L,SchoolSizeIndicator_S,SchoolSizeIndicator_S-M
1320,0.0,1.0,384.0,19.0,1.0,0.475,3.0,2.0,407.0,0.200,...,0,0,0,1,0,0,0,0,0,1
1412,0.0,5.0,2117.0,14.0,2.0,0.667,0.0,2.0,184.0,0.286,...,0,0,0,1,0,1,0,0,1,0
530,0.0,5.0,736.0,29.0,6.0,0.460,4.0,4.0,500.0,0.111,...,0,0,0,1,0,0,0,0,0,1
554,0.0,5.0,1799.0,13.0,2.0,0.929,3.0,1.0,761.0,0.071,...,0,0,0,1,0,1,0,1,0,0
229,0.0,3.0,1058.0,19.0,1.0,0.704,1.0,5.0,334.0,0.185,...,0,0,1,0,0,0,0,0,1,0


En este punto, los datos de entrenamiento y prueba tienen las mismas características y están codificados de la misma forma basado en la información de entrenamiento. Con lo anterior, es posible entrenar los modelos que queramos para predicción.

# Modelamiento

En esta sección se realizará
- Línea Base
- Support Vector Machine

## Línea Base -- Sencilla

Lo ideal al resolver un problema de Machine Learning es siempre tener una línea base de comparación. Usualmente se utiliza un modelo previo, alguna heurística,el modelo más sencillo al alcance o lo que hubiera hecho un humano. En este caso contamos con información de una variable binaria que podría ser un muy buen predictor de no reserva el siguiente año: 

> `Is.Non.Annual.`: Variable binaria que indica si el grupo suele saltarse un año entre programas de viaje. Aquellos que se suelen saltar el año, muy rara vez viajan el año siguiente. 

Utilice esta variable para construir una línea base de comparación.

In [31]:
### Determine qué proporción de observaciones tienen 1 en este campo

X_train['Is.Non.Annual.'].value_counts()/len(X_train)

### ¿Qué puede decir de su resultado?

0.0    0.846292
1.0    0.153708
Name: Is.Non.Annual., dtype: float64

In [ ]:
### Utilice esta información para crear una predicción (sin un modelo) -> Train y Test

###

A continuación deberemos revisar el desempeño de nuestra línea base. Para ello, sugerimos utilizar las siguientes herramientas:

- [`sklearn.metrics.accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)
- [`sklearn.metrics.classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)
- [`sklearn.metrics.confusion_matrix`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)
- [`sklearn.metrics.ConfusionMatrixDisplay`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html)

In [ ]:
### Evalúe el desempeño de su modelo en train -- Accuracy, Matriz de Confusión, F1, Precisión, Recall...

None

###


In [ ]:
### Evalúe el desempeño de su modelo en test -- Accuracy, Matriz de Confusión, F1, Precisión, Recall...

None

###


¿Qué conclusiones puede formar respecto a la línea base? ¿Es una buena línea base?

## Support Vector Machine -- Default

En esta sección se entrenará un modelo de SVM utilizando los parámetros por defcto del modelo. Siempre es bueno conocer cómo se desempeña el modelo sin optimizar hiperparámetros para determinar su desempeño "base".

Para esto necesitaremos utilizar

- [`sklearn.svm.SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html): Permitirá entrenar el modelo SVM
- [`sklearn.preprocessing.MinMaxScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html): Para pre-procesamiento (básico) de features.
- [`sklearn.pipeline.Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): Para encadenamiento de pasos


In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

np.random.seed(10)

# Encadene los pasos de Pre-Procesamiento y Modelo

###

model_steps = None
model_steps.append(('MinMaxScaling', None)) # escalamos los datos
model_steps.append(('SVM', None)) # definimos SVM(probability = True)

###


# Defina el Modelo utilizando el Pipeline
clf_default = Pipeline(model_steps, verbose=False)

In [ ]:
# Ajuste el Modelo en los Datos de Entrenamiento
###

None

###

En este momento se tiene un modelo entrenado con los parámetros por defecto. A continuación se realizarán las predicciones y la evaluación del desempeño del modelo. 

La predicción se realiza utilizando los métodos `clf_deafult.predict` para la clase (1 o 0), y `clf_default.predict_proba` para predecir las probabilidades. Tenga en cuenta que al utilizar `predict_proba` se obtienen dos columnas, una con la probabilidad para la clase `0` y otra para la clase `1`.  

In [ ]:
# Realice la Predicción de las clases

###

y_pred_default = None

###


# Realice la predicción de las probabilidades

###
y_pred_default_proba = None

###

## Métricas de Evaluación

De igual forma que con la línea base, es necesario realizar la evaluación del desempeño del modelo en train y en test. Adicionalmente a las métricas anteriores, se pide la evaluación utilizando la curva ROC y el AUC. 

- [`sklearn.metrics.roc_curve`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html)
- [`sklearn.metrics.roc_auc_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)

In [ ]:
# Evalúe el desempeño de su modelo en train -- Accuracy, Matriz de Confusión, F1, Precisión, Recall...


###
None


###

In [ ]:
# Realice el cálculo de la curva ROC

### 


false_positives_rate, true_positives_rate, threshold = None

###

In [ ]:
# Grafique la curva ROC

plt.title('ROC - SVM Lineal (Default)')
###


plt.plot(None, None, label = 'SVM Default')


###

plt.plot([0, 1], ls="--")
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.ylim(0, 1)
plt.xlim(0, 1)
plt.legend()
plt.show()

In [ ]:
# Calcule el AUC del Modelo

###

print('AUC Score: ', None)

###

¿Qué puede concluir de este modelo frente a la línea base?

# Utilización de Kernels -- Búsqueda de Hiperparámetros

No siempre el modelo con los parámetros por defecto suele ser el mejor. Sin embargo, sí suele dar un indicio del desempeño que se va a conseguir con el modelo. 

Una vez construída la línea base, se suele realizar la estimación utilizando una búsqueda de hiperparámetros. Esta búsqueda permite seleccionar los hiperparámetros óptimos para el modelo, de forma tal que genere el mejor desempeño posible.

## Búsqueda de la Mejor Combinación

Existen varias estrategias de búsqueda de hiperparámetros. En esta sección se utilizará principalmente [`sklearn.model_selection.GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html), que nos permite definir una "malla" de parámetros sobre los cuales entrenar modelos. 

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Defina los hiperparametros que quiere probar

###

hyperparameters_grid = [{
    'SVM__kernel': [None],  # Una lista con los kernels que desea probar
    'SVM__C': [None], # Una lista con los valores de C para regularización
    'SVM__gamma': [None] # Una lista con los valores de gamma de los kernels
}]

###

In [ ]:
Hyper_SVM = Pipeline(model_steps, verbose=False)

In [ ]:
grid_search = GridSearchCV(Hyper_SVM, hyperparameters_grid, cv=5,  n_jobs=3)

In [ ]:
grid_search.fit(X_train, y_train)

### Selección del Mejor Modelo

En este punto se entrenaron tantos modelos como combinaciones de hiperparámetros definidos. Solo uno de ellos puede ser el mejor modelo. A continuación, obtenemos el mejor modelo de los entrenados y evaluamos su desempeño en test.

In [ ]:
cv_performance = grid_search.best_score_
test_performance = grid_search.score(X_test, y_test)

In [ ]:
print('Best parameter: {}'.format(str(grid_search.best_params_)))
print('Cross-validation accuracy score: {0:0.3f}'.format(cv_performance))

## Entrenamiento del Mejor Modelo

El mejor modelo fue entrenado utilizando Cross-Validation, por lo que no tenemos un modelo entrenado utilizando la totalidad de los datos de entrenamiento. Por esta razón, tomamos los hiperparámetros del mejor modelo y entrenamos uno nuevo.

In [ ]:
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

Una vez entrenado, ya es posible realizar las predicciones y evaluar su desempeño para comparación con los modelos anteriores.

In [ ]:
y_pred_hp = best_model.predict(X_test)
y_pred_prob_hp = best_model.predict_proba(X_test)[:,1]

In [ ]:
 print('AUC Score: ', roc_auc_score(y_test, y_pred_prob_hp))

In [ ]:
false_positives_rate_hp, true_positives_rate_hp, threshold = roc_curve(y_test, y_pred_prob_hp)

In [ ]:
# Grafique la curva ROC para ambos modelos


plt.title('ROC - Comparación Modelos')

###

plt.plot(None, None, label = 'SVM Default')
plt.plot([0, 1], ls="--")
plt.plot(None, None, label = 'Tuned SVM')

###

plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.ylim(0, 1)
plt.xlim(0, 1)
plt.show()

Escoger el mejor modelo:
Linea base.
SVC.
Reg Logi.

Mini informe "Crisp-DM"

# Interpretación de Resultados

¿Qué puede decir respecto a los resultados obtenidos de la línea base y el SVM? ¿Cómo impactan sus resultados al negocio de STC?